In [21]:
# CODE TO RETRIEVE DATA FROM THE WEATHER API AND STORE IT IN A CSV FILE
# FROM 2019/01/01 TO 2022/12/31

import requests
import time
import datetime as dt
import csv 

key = 'nEqafbclC25HU3MXq5DBB8YqNYIqBnck4jJDrEdI'

start_date = dt.date(2021, 1, 1)
end_date = dt.date(2023, 12, 31)

assert start_date < end_date, 'Start date must be before end date'

delta = dt.timedelta(days=1)

freq = 1/20

day = start_date

columns = ['timestamp', 'mm_precip', 'temperature']

with open('data/weather_data/weather_2.csv', mode = 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()

    while day <= end_date:

        day_string = day.strftime('%Y/%m/%d')

        url = 'https://api.meteo.cat/xema/v1' + '/estacions/mesurades/X8/' + day_string 

        print(day_string)

        headers = {'Accept': 'application/json', 'X-API-KEY': key}

        data = requests.get(url, headers=headers).json()#[0]

        # the codi variables, 35 and 32, correspond to precipitation and temperature respectively. The json file retrieved
        # contains information on many more variables, but we are only interested in these two.

#        precipitation = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 35][0]
#        temperature = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 32][0]
#
 #       date_variables = [{'timestamp':int(dt.datetime.strptime(d['data'], '%Y-%m-%dT%H:%MZ').timestamp()), 'mm_precip':d['valor']} for d in precipitation]
#
 #       for i in range(len(date_variables)):
  #          date_variables[i]['temperature'] = temperature[i]['valor']
#
 #       writer.writerows(date_variables)
        day += delta

        time.sleep(freq)



2021/01/01
2021/01/02
2021/01/03
2021/01/04
2021/01/05
2021/01/06
2021/01/07
2021/01/08
2021/01/09
2021/01/10
2021/01/11
2021/01/12
2021/01/13


KeyboardInterrupt: 

In [23]:
day_string = '2020/01/25'
url = 'https://api.meteo.cat/quotes/v1' + '/consum-actual/' #+ day_string 
requests.get(url, headers=headers).json()

{'client': {'nom': '477 Alejandro Jaramillo Pazmiño UB'},
 'plans': [{'nom': 'XEMA_750 OD',
   'periode': 'Mensual',
   'maxConsultes': 750,
   'consultesRestants': 0,
   'consultesRealitzades': 750},
  {'nom': 'Quota',
   'periode': 'Mensual',
   'maxConsultes': 300,
   'consultesRestants': 300,
   'consultesRealitzades': 0},
  {'nom': 'Referència Bàsic',
   'periode': 'Mensual',
   'maxConsultes': 2000,
   'consultesRestants': 2000,
   'consultesRealitzades': 0},
  {'nom': 'Predicció_100',
   'periode': 'Mensual',
   'maxConsultes': 100,
   'consultesRestants': 100,
   'consultesRealitzades': 0},
  {'nom': 'XDDE_250',
   'periode': 'Mensual',
   'maxConsultes': 250,
   'consultesRestants': 250,
   'consultesRealitzades': 0}]}